# Single Variable Analysis
---
### Notebook for Single Variable Analysis in all three processed datasets

Creation: 07.02.2021

## Required Libraries
---

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import textwrap
import json
import os

## Constants
---

In [6]:
PATH = {}
PATH["data_raw"] = "../data/raw/"
PATH["data_interim"] = "../data/interim/"
PATH["data_processed"] = "../data/processed/"
PATH["data_external"] = "../data/external/"
PATH["references"] = "../data/references/"

FILENAME = {}
FILENAME["accidents"] = "Road Safety Data - Accidents 2019.csv"
FILENAME["casualties"] = "Road Safety Data - Casualties 2019.csv"
FILENAME["vehicles"] = "Road Safety Data- Vehicles 2019.csv" # the original dataset has a small typing mistake
FILENAME["variable_lookup"] = "variable lookup.xls"

SUMMARY = {}

TABLENAMES = ["accidents", "casualties", "vehicles"]

## Importing Processed Datasets and Variable Lookups
---

In [7]:
DATA_LEEDS = {}
for dataset in TABLENAMES:
    DATA_LEEDS[dataset] = pd.read_csv(PATH['data_processed'] + FILENAME[dataset])

In [8]:
xls = pd.ExcelFile(PATH['references'] + FILENAME['variable_lookup'])

# read in excel data into python dict of dicts
excel_dict = {i: xls.parse(xls.sheet_names[i]).to_dict() for i in range(len(xls.sheet_names))}

# create convenient lookup dictionary for all excel sheets
VARIABLE_LOOKUP = {}
for x in range(2, len(excel_dict)):
    VARIABLE_LOOKUP[x] = {}
    for title in [('code', 'label'), ('Code', 'Label')]:
        try:
            for i in range(len(excel_dict[x][title[0]])):
                VARIABLE_LOOKUP[x][excel_dict[x][title[0]][i]] = excel_dict[x][title[1]][i]
            continue
        except: None

In [9]:
del VARIABLE_LOOKUP[36][' M']
VARIABLE_LOOKUP[36][-1] = 'Undefined'

## Lookup and Variable Inspection¶
---

To efficiently analyse and plot singular attributes in the datasets, we need to keep track...
- ...of the column index of the column
- ...of the type of the variable (numerical, categorical)
- ...of the variable look-up for categorical data (those are taken from 'data/references/variable\ lookup.xls')

We therefore create a lookup dictionary for each of the datasets for a code efficient analysis.

In [10]:
LABELS = {}
LABELS['accidents'] = [5, 6, 10, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30]
LABELS['casualties'] = [3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
LABELS['vehicles'] = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 18]

PLOTTING = {}
PLOTTING['accidents'] = [None, None, None, None, None, 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', None]
PLOTTING['casualties'] = [None, None, None, 'bar', 'bar', 'hist', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar',None]
PLOTTING['vehicles'] = [None, None, 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'bar', 'hist', 'bar', 'hist', 'bar', 'hist', None, 'bar', None]

FIVENUM = {}
FIVENUM['accidents'] = [False, False, False, False, False, False, True, True, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
FIVENUM['casualties'] = [False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False]
FIVENUM['vehicles'] = [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, True, False, False, False]

### Accidents
---

In [11]:
def initialise_summary(dataset, labels, plotting, fivenum, start_at=0):
    # initialise the lookup dictionary with the column name and variable type
    SUMMARY[dataset] = {}
    for i in range(DATA_LEEDS[dataset].shape[1]):
        SUMMARY[dataset][i] = {'Name': list(DATA_LEEDS[dataset])[i]}
        if plotting[i] == 'bar':
            SUMMARY[dataset][i].update({'Plot': 'bar'})
        elif plotting[i] == 'hist':
            SUMMARY[dataset][i].update({'Plot': 'hist'})
        else: SUMMARY[dataset][i].update({'Plot': None})

        if fivenum[i] == True:
            SUMMARY[dataset][i].update({'Summary': True})
        else: SUMMARY[dataset][i].update({'Summary': False})

    # add the maps to the lookup dictionary
    categorical_counter = 0
    for column in labels:
        if dataset == 'casualties':
            if categorical_counter == 2: 
                SUMMARY[dataset][column]['Map'] = VARIABLE_LOOKUP[35]
                categorical_counter += 1
                continue
            if categorical_counter == 10:
                SUMMARY[dataset][column]['Map'] = VARIABLE_LOOKUP[48]
                categorical_counter += 1
                continue
            if categorical_counter == 11:
                SUMMARY[dataset][column]['Map'] = VARIABLE_LOOKUP[47]
                categorical_counter += 1
                continue
        
        SUMMARY[dataset][column]['Map'] = VARIABLE_LOOKUP[start_at+categorical_counter]
        categorical_counter += 1

In [12]:
for i, start_at in zip(list(range(3)), [2, 37, 22]):
    initialise_summary(TABLENAMES[i], LABELS[TABLENAMES[i]], PLOTTING[TABLENAMES[i]], FIVENUM[TABLENAMES[i]], start_at=start_at)

In [18]:
VARIABLE_LOOKUP[4]

{1: 'Sunday',
 2: 'Monday',
 3: 'Tuesday',
 4: 'Wednesday',
 5: 'Thursday',
 6: 'Friday',
 7: 'Saturday'}

In [9]:
SUMMARY['casualties'][12]

{'Name': 'Pedestrian_Road_Maintenance_Worker',
 'Plot': 'bar',
 'Summary': False,
 'Map': {0: 'No / Not applicable',
  1: 'Yes',
  2: 'Not Known',
  -1: 'Data missing or out of range'}}

In [10]:
# time mapping
SUMMARY['accidents'][11]['Map'] = {i: f"{i}-{i+1}" for i in range(24)}

# date mapping
months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
SUMMARY['accidents'][9]['Map'] = {i+1: months[i] for i in range(12)}

In [11]:
SUMMARY['accidents'][9]

{'Name': 'Date',
 'Plot': 'bar',
 'Summary': False,
 'Map': {1: 'JAN',
  2: 'FEB',
  3: 'MAR',
  4: 'APR',
  5: 'MAY',
  6: 'JUN',
  7: 'JUL',
  8: 'AUG',
  9: 'SEP',
  10: 'OCT',
  11: 'NOV',
  12: 'DEC'}}

## Single Variable Analysis
---

In [12]:
def get_uniques_and_counts(data):
    uniques, counts = np.unique(data, return_counts=True)

    return uniques, counts

In [13]:
def get_fivenumsummary(data):
    return np.percentile(data[data >= 0], [0, 25, 50, 75, 100])

In [14]:
def compute_summary_categorical(summary, data): #
    for column in range(len(summary)):
        uniques, counts = get_uniques_and_counts(data[summary[column]['Name']])
        summary[column]['No_Uniques'] = len(uniques) # get the number of uniques for each variable
            
        if summary[column]['Plot'] == 'bar': # get the counts for each uniques for all categorical attributes
            if len(uniques) < 100:
                summary[column]['Uniques'] = {uniques[i]: counts[i] for i in range(len(uniques))}

In [15]:
def compute_summary_numerical(summary, data):
    for column in range(len(summary)):
        if summary[column]['Plot'] == 'hist' or summary[column]['Summary'] == True:
            summary[column]['Data'] = np.array(data[summary[column]['Name']])

In [16]:
def compute_fivenumbersummary(summary, data):
    for column in range(len(summary)):
        if summary[column]['Summary'] == True:
            summary[column]['Five_Number_Summary'] = get_fivenumsummary(data[summary[column]['Name']])

In [17]:
for dataset in TABLENAMES:
    compute_summary_categorical(SUMMARY[dataset], DATA_LEEDS[dataset])
    compute_summary_numerical(SUMMARY[dataset], DATA_LEEDS[dataset])
    compute_fivenumbersummary(SUMMARY[dataset], DATA_LEEDS[dataset])

## Saving Numerical Report
---

In [18]:
def save_numerical_report(summary, name, save_to='csv'):
    summary_dataframe = pd.DataFrame(summary)

    if save_to == 'csv':
        try: os.makedirs('../reports/numerical_summaries/csv')
        except: None
        
        summary_dataframe.to_csv(f'../reports/numerical_summaries/csv/numerical_summary_{name}.csv')
        
    elif save_to == 'json':
        
        try: os.makedirs('../reports/numerical_summaries/json')
        except: None

        summary_dataframe.to_json(f'../reports/numerical_summaries/json/numerical_summary_{name}.json')

In [19]:
for i in range(len(TABLENAMES)):
    save_numerical_report(SUMMARY[TABLENAMES[i]], TABLENAMES[i], save_to='csv')
    save_numerical_report(SUMMARY[TABLENAMES[i]], TABLENAMES[i], save_to='json')

## Visualisation
---
In this section, we are visualising the results of the single variable analysis in all three datasets - both within the Jupyter and per export into 'reports/figures' for later use.

In [20]:
def visualise_categorical(data, keep_missing_values=True):
    # create figure and axes (with padding for better exporting)
    fig = plt.figure(figsize=(32,18))
    ax = fig.add_axes([.15,.15,.7,.7])
    
    # variables depending on missing_values variable
    if keep_missing_values:
        x = list(data['Uniques'].keys())
        y = list(data['Uniques'].values())
        title = title = f"Distribution: {data['Name'].replace('_', ' ')} (with missing values)"
        color = 'darkred'
        yticks = list(data['Uniques'].keys())
    
    else: 
        if -1 in list(data['Uniques'].keys()):
            x = list(data['Uniques'].keys())[1:]
            y = list(data['Uniques'].values())[1:]
            yticks = list(data['Uniques'].keys())[1:]
        else: 
            x = list(data['Uniques'].keys())
            y = list(data['Uniques'].values())
            yticks = list(data['Uniques'].keys())

        title = f"Distribution: {data['Name'].replace('_', ' ')} (without missing values)"
        color = 'darkblue'
        
    spaced_ticks = [i for i in range(len(yticks))]
    # plot bar 
    ax.barh(spaced_ticks, y, align='center', color=color)

    # set title
    ax.set_title(title, fontweight='bold')

    # format axis-labels
    ax.set_xlabel('Number of Accidents')
    try: # account for 0% datasets
        ax.set_xlim(0, 1.15*max(y)) 
    except: None

    ax.set_yticks(spaced_ticks)
    ax.tick_params(axis='y', which='major', pad=10)
    ax.invert_yaxis()

    try: 
        y_labels = [data['Map'][i] for i in x] # use lookup from xls 
        ax.set_yticklabels([textwrap.fill(label, 10) for label in y_labels])
    except: None # account for variables that do not have lookup mapping

    # insert counts and percentages as text next to the corresponding bars
    for x_cord, y_cord in zip(spaced_ticks,y):
        ax.text(y_cord, x_cord, f'{y_cord} ({str(100*round(y_cord/sum(y), 3))[:5]}%)' , color='black', fontweight='bold')
    
    plt.tight_layout()


    return fig

In [21]:
def visualise_numerical(data, keep_missing_values=True):
    # create figure and axes (with padding for better exporting)
    fig = plt.figure(figsize=(16,9))
    ax = fig.add_axes([.1,.1,.8,.8])

    if keep_missing_values:
        title = f"Distribution: {data['Name'].replace('_', ' ')} (with missing values)"
        data = data['Data']
        color = 'darkred'

    else: 
        title = f"Distribution: {data['Name'].replace('_', ' ')} (without missing values)"
        data = data['Data'][(data['Data'] != -1)] # masking out -1
        color = 'darkblue'

    ax.hist(data, bins=50, color=color)

    # set title
    ax.set_title(title, fontweight='bold')

    # format axis-labels
    ax.set_ylabel('Number of Accidents')
    ax.set_xlabel('Age')

    return fig

In [22]:
def visualise_fivenumsummary(data):
    # create figure and axes (with padding for better exporting)
    fig = plt.figure(figsize=(16,9))
    ax = fig.add_axes([.1,.1,.8,.8])

    ax.boxplot(data['Data']);

    ax.set_title(f"Boxplot of {data['Name'].replace('_', ' ')}", fontweight='bold')

    return fig

## Saving Figures
---

In [ ]:
def save_figure(figure, index, name, dataset, keep_missing_values=True, boxplot=False):
    if boxplot==False:
        try:
            if keep_missing_values: 
                os.makedirs(f'../reports/figures/{dataset}/with_missing_values')
            else: 
                os.makedirs(f'../reports/figures/{dataset}/no_missing_values')
        except: None

        if keep_missing_values:
            figure.savefig(f'../reports/figures/{dataset}/with_missing_values/{index}_{name}.pdf')
        else: 
            figure.savefig(f'../reports/figures/{dataset}/no_missing_values/{index}_{name}.pdf')

    else:
        try: 
            os.makedirs(f'../reports/figures/{dataset}/boxplots')
        except: None
    
        figure.savefig(f'../reports/figures/{dataset}/boxplots/{index}_{name}.pdf')

In [ ]:
def save_all(data, name, missing_values):
    for column in range(len(data)):
        if data[column]['Plot'] == 'bar':
            fig = visualise_categorical(data[column], keep_missing_values=missing_values)

            save_figure(fig, column, data[column]['Name'], name, missing_values)
        elif data[column]['Plot'] == 'hist':
            fig = visualise_numerical(data[column], keep_missing_values=missing_values)

            save_figure(fig, column, data[column]['Name'], name, keep_missing_values=missing_values)
        if data[column]['Summary']:
            fig = visualise_fivenumsummary(data[column])

            save_figure(fig, column, data[column]['Name'], name, boxplot=True)

In [ ]:
%%capture # prevent plots from showing inline in jupyter
for dataset in TABLENAMES:
    save_all(SUMMARY[dataset], dataset, missing_values=True)
    save_all(SUMMARY[dataset], dataset, missing_values=False)